In [ ]:
import pandas as pd
# Load data
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/cars (1).csv')

In [ ]:
data.isnull().sum()

Make                    0
Model                   0
Year                    0
Engine Fuel Type        3
Engine HP              69
Engine Cylinders       30
Transmission Type       0
Driven_Wheels           0
Number of Doors         6
Market Category      3742
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64

In [ ]:
data

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [ ]:
X=data.copy()
y=X.pop('MSRP')

In [ ]:
X

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204


In [ ]:
y

0        46135
1        40650
2        36350
3        29450
4        34500
         ...  
11909    46120
11910    56670
11911    50620
11912    50920
11913    28995
Name: MSRP, Length: 11914, dtype: int64

In [ ]:
X['Vehicle Size'].value_counts()

Compact    4764
Midsize    4373
Large      2777
Name: Vehicle Size, dtype: int64

First of all I will handle the missing values using imputation with different strategies :

In [ ]:

cols_with_missing=[col for col in X if X[col].isnull().any()]

cols_with_missing


['Engine Fuel Type',
 'Engine HP',
 'Engine Cylinders',
 'Number of Doors',
 'Market Category']

In [ ]:
X[cols_with_missing].dtypes

Engine Fuel Type     object
Engine HP           float64
Engine Cylinders    float64
Number of Doors     float64
Market Category      object
dtype: object

In [ ]:
numeric_x=X.select_dtypes(exclude=['object'])

from sklearn.impute import SimpleImputer

my_imputer1=SimpleImputer(strategy='mean')
numeric_x_imputed=pd.DataFrame(my_imputer1.fit_transform(numeric_x))

In [ ]:
categorical_x=X.select_dtypes(exclude=['int64','float64'])

my_imputer2=SimpleImputer(strategy='most_frequent')
categorical_x_imputed=pd.DataFrame(my_imputer2.fit_transform(categorical_x))

In [ ]:
numerical_cols=[col for col in X if X[col].dtypes=='float64' or X[col].dtypes=='int64']
categorical_cols=[col for col in X if X[col].dtypes=='object']

In [ ]:
numeric_x_imputed.columns=numerical_cols
categorical_x_imputed.columns=categorical_cols

In [ ]:
numeric_x_imputed

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity
0,2011.0,335.0,6.0,2.0,26.0,19.0,3916.0
1,2011.0,300.0,6.0,2.0,28.0,19.0,3916.0
2,2011.0,300.0,6.0,2.0,28.0,20.0,3916.0
3,2011.0,230.0,6.0,2.0,28.0,18.0,3916.0
4,2011.0,230.0,6.0,2.0,28.0,18.0,3916.0
...,...,...,...,...,...,...,...
11909,2012.0,300.0,6.0,4.0,23.0,16.0,204.0
11910,2012.0,300.0,6.0,4.0,23.0,16.0,204.0
11911,2012.0,300.0,6.0,4.0,23.0,16.0,204.0
11912,2013.0,300.0,6.0,4.0,23.0,16.0,204.0


In [ ]:
categorical_x_imputed

,Make,Model,Engine Fuel Type,Transmission Type,Driven_Wheels,Market Category,Vehicle Size,Vehicle Style
0,BMW,1 Series M,premium unleaded (required),MANUAL,rear wheel drive,"Factory Tuner,Luxury,High-Performance",Compact,Coupe
1,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Convertible
2,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,High-Performance",Compact,Coupe
3,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Coupe
4,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,Luxury,Compact,Convertible
...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11910,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11911,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11912,Acura,ZDX,premium unleaded (recommended),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback


In [ ]:
X_new=pd.concat([numeric_x_imputed,categorical_x_imputed],axis=1)

In [ ]:
X_new

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,Make,Model,Engine Fuel Type,Transmission Type,Driven_Wheels,Market Category,Vehicle Size,Vehicle Style
0,2011.0,335.0,6.0,2.0,26.0,19.0,3916.0,BMW,1 Series M,premium unleaded (required),MANUAL,rear wheel drive,"Factory Tuner,Luxury,High-Performance",Compact,Coupe
1,2011.0,300.0,6.0,2.0,28.0,19.0,3916.0,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Convertible
2,2011.0,300.0,6.0,2.0,28.0,20.0,3916.0,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,High-Performance",Compact,Coupe
3,2011.0,230.0,6.0,2.0,28.0,18.0,3916.0,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Coupe
4,2011.0,230.0,6.0,2.0,28.0,18.0,3916.0,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,Luxury,Compact,Convertible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,2012.0,300.0,6.0,4.0,23.0,16.0,204.0,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11910,2012.0,300.0,6.0,4.0,23.0,16.0,204.0,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11911,2012.0,300.0,6.0,4.0,23.0,16.0,204.0,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11912,2013.0,300.0,6.0,4.0,23.0,16.0,204.0,Acura,ZDX,premium unleaded (recommended),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback


In [ ]:
X_new.isnull().sum().sum()

0

After filling the missing values , I will use some feature engineering aproaches ,starting with finding the MI :

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:

for colname in X_new.select_dtypes("object"):
    X_new[colname], _ = X_new[colname].factorize()

discrete_features = X_new.dtypes == int


In [ ]:
mi_scores = mutual_info_regression(X_new, y, discrete_features=discrete_features)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X_new.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
mi_scores


Model                2.031799
Engine HP            1.228091
Market Category      0.734251
Make                 0.731845
Popularity           0.631246
Year                 0.547724
city mpg             0.402920
Vehicle Style        0.381454
highway MPG          0.368289
Engine Cylinders     0.362255
Engine Fuel Type     0.359833
Driven_Wheels        0.261524
Vehicle Size         0.198956
Transmission Type    0.180653
Number of Doors      0.114774
Name: MI Scores, dtype: float64

In [ ]:
# I can create new feature here , I searched fow a way to combine city MPG and highway MPG
# so I will add a new feature
# you can ask a domain expert to find the formula, in my case I used Google


X_new['Combined_MPG']=  X_new['city mpg'] * .55 +  X_new['highway MPG'] * .45

In [ ]:
X_new['Combined_MPG']

0        22.15
1        23.05
2        23.60
3        22.50
4        22.50
         ...  
11909    19.15
11910    19.15
11911    19.15
11912    19.15
11913    21.05
Name: Combined_MPG, Length: 11914, dtype: float64

In [ ]:
# I will do new MI serie, if my new feature gets high MI , I will keep it

for colname in X_new.select_dtypes("object"):
    X_new[colname], _ = X_new[colname].factorize()

discrete_features = X_new.dtypes == int



mi_scores = mutual_info_regression(X_new, y, discrete_features=discrete_features)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X_new.columns)
mi_scores = mi_scores.sort_values(ascending=False)


mi_scores

Model                2.051452
Engine HP            1.222691
Make                 0.739900
Market Category      0.732838
Popularity           0.636025
Combined_MPG         0.630339
Year                 0.547088
city mpg             0.398711
Vehicle Style        0.382759
highway MPG          0.370542
Engine Cylinders     0.364777
Engine Fuel Type     0.359315
Driven_Wheels        0.261694
Vehicle Size         0.196062
Transmission Type    0.177548
Number of Doors      0.112515
Name: MI Scores, dtype: float64

In [ ]:
# another thing I can do is to remove 'number of doors' feature since it has the least MI
# which makes sense because you can conclude the number of doors by other features 'vehicle size' and 'vehicle style'


X_new.drop('Number of Doors',axis=1,inplace=True)

In [ ]:
X_new

,Year,Engine HP,Engine Cylinders,highway MPG,city mpg,Popularity,Make,Model,Engine Fuel Type,Transmission Type,Driven_Wheels,Market Category,Vehicle Size,Vehicle Style,Combined_MPG
0,2011.0,335.0,6.0,26.0,19.0,3916.0,0,0,0,0,0,0,0,0,22.15
1,2011.0,300.0,6.0,28.0,19.0,3916.0,0,1,0,0,0,1,0,1,23.05
2,2011.0,300.0,6.0,28.0,20.0,3916.0,0,1,0,0,0,2,0,0,23.60
3,2011.0,230.0,6.0,28.0,18.0,3916.0,0,1,0,0,0,1,0,0,22.50
4,2011.0,230.0,6.0,28.0,18.0,3916.0,0,1,0,0,0,3,0,1,22.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,2012.0,300.0,6.0,23.0,16.0,204.0,35,913,0,1,2,70,1,4,19.15
11910,2012.0,300.0,6.0,23.0,16.0,204.0,35,913,0,1,2,70,1,4,19.15
11911,2012.0,300.0,6.0,23.0,16.0,204.0,35,913,0,1,2,70,1,4,19.15
11912,2013.0,300.0,6.0,23.0,16.0,204.0,35,913,2,1,2,70,1,4,19.15


Now I will build the model and then use MAE for validation :

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [ ]:
def get_MAE(e):
  model=RandomForestRegressor(n_estimators=e,random_state=0)
  X_train,X_val,y_train,y_val=train_test_split(X_new,y,train_size=.8,test_size=.2,random_state=0)
  model.fit(X_train,y_train)
  yp=model.predict(X_val)
  error=mean_absolute_error(yp,y_val)
  return error



Now I will try the model parameters to find the least error   :

(I will try different number of estimators and then validate the model using the cross validation function)

In [ ]:
get_MAE(50)

3223.892129604184

In [ ]:
for i in [25,50,75,100]:
  print(get_MAE(i))

3187.0396779408647
3223.892129604184
3198.5672699779543
3181.736281528204


In [ ]:
for i in [100,200,300]:
  print(get_MAE(i))

3181.736281528204
3165.4667312903953
3148.450403048192


In [ ]:
for i in [300,350,400]:
  print(get_MAE(i))

3148.450403048192
3134.5618690906167
3132.083507468915


In [ ]:
#The last MAE before creating the final model
get_MAE(400)

3132.083507468915

In [ ]:
# The best number of estimators is 400

Now,I can build my final model and train it in all of the data !

In [ ]:

myFinalModel=RandomForestRegressor(n_estimators=400,random_state=0)
myFinalModel.fit(X_new,y)

RandomForestRegressor(n_estimators=400, random_state=0)

In [ ]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(myFinalModel, open(filename, 'wb'))